## Домашняя работа №2 (часть 1)

Перед началом работы включу проверку PEP-8. Она закомментированна (перед этим я учла все ее замечания),
потому что из-за нее все перестает работать как надо. 

In [1]:
# %load_ext pycodestyle_magic
# %pycodestyle_on

#### Лемматизирую и анализирую текст книги в Майстеме
Анализирую текст, замеряю время, потраченное на анализ. Лемматизированный текст пригодится для дальнейших действий.

In [2]:
from pymystem3 import Mystem 

with open('tragedy.txt', 'r', encoding='utf-8') as f:
    text = ' '.join(f.read().split('\n'))

m = Mystem()
lemmas = m.lemmatize(text)
lemma_text = ''.join(lemmas)

%time analyzed_mystem = m.analyze(text)

Wall time: 20.2 s


#### Сохраняю результат анализа в файле mystem_data.json

In [3]:
import json

with open('mystem_data.json', 'w', encoding='utf-8') as f:
    json.dump(analyzed_mystem, f, ensure_ascii = False)

#### Токенизирую с помощью NLTK
Также удаляю все символы, не являющиеся словами. Измеряю время.
Токенизируется уже лемматизированный текст.

In [4]:
import nltk
from nltk.tokenize import word_tokenize

%time words = [w for w in word_tokenize(lemma_text) if w.isalpha()]

Wall time: 2.75 s


#### Анализирую с помощью Pymorphy
Снова измеряю время.

In [5]:
%%time

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

analyzed = []
for word in words:
    analyzed.append(morph.parse(word))

Wall time: 1min 14s


#### Теперь подсчитаем проценты всех частей речи
Похоже, что INFN = VERB, так как обрабатывались леммы.

In [6]:
from collections import Counter

pos = []
for ana in analyzed:
    pos.append(ana[0].tag.POS)

count_pos = dict(Counter(pos))

percent_dict = {}
for part in list(count_pos):
    percent = count_pos[part] / len(words) * 100
    percent_dict[part] = percent
    
for key, value in percent_dict.items():
    print(key, value, "%", end='. ')

NOUN 63263
ADJF 32878
None 183
CONJ 37136
PREP 28052
ADVB 21602
PRCL 18926
NUMR 1500
INFN 49232
NPRO 28415
COMP 178
VERB 592
ADJS 588
GRND 169
PRTF 412
PRED 1696
INTJ 163
NOUN 22.19871221292349 %. ADJF 11.536747548116567 %. None 0.06421390599505238 %. CONJ 13.030861273400355 %. PREP 9.843325087285296 %. ADVB 7.580048072705581 %. PRCL 6.6410512834008815 %. NUMR 0.5263434917627243 %. INFN 17.275295190974962 %. NPRO 9.970700212291876 %. COMP 0.06245942768917662 %. VERB 0.20773023141568855 %. ADJS 0.20632664877098797 %. GRND 0.05930136673860028 %. PRTF 0.14456901240416162 %. PRED 0.5951190413530536 %. INTJ 0.057195992771549374 %. 

#### Напишу функцию для составления частотного словаря и найду топ-20 глаголов и наречий

In [7]:
def search_pos(data, pos):
    n_list = []
    for ana in data:
        if ana[0].tag.POS == pos:
            n_list.append(ana[0].word)
    return Counter(n_list).most_common(20)


top_verbs = search_pos(analyzed, 'INFN')
top_v = [x[0] + ': ' + str(x[1]) for x in top_verbs]
print(', '.join(top_v))

top_advs = search_pos(analyzed, 'ADVB')
top_a = [x[0] + ': ' + str(x[1]) for x in top_advs]
print(', '.join(top_a))

быть: 4525, сказать: 1001, знать: 887, думать: 644, говорить: 635, хотеть: 598, понимать: 558, становиться: 535, видеть: 438, отвечать: 369
теперь: 771, очень: 668, здесь: 565, уже: 556, потом: 486, тут: 418, тогда: 404, много: 384, хорошо: 375, где: 367


#### Составлю топ-25 биграмм
Большинство из них представляет собой комбинацию из предлогов, частиц, личных местоимений, модальных глаголов, то есть наиболее частотных слов в языке. Также из этих данных можно предположить, чтл Клайд - главный герой книги. 

In [8]:
bigs = nltk.bigrams(words)
top_bigrams = Counter(bigs).most_common(25)
for pair in top_bigrams:
    print(pair[0][0], pair[0][1] + ':', pair[1], end='. ')

не мочь: 607. он не: 564. что он: 564. и не: 426. в этот: 408. мочь быть: 407. он быть: 371. то что: 343. и он: 341. я не: 341. не быть: 336. он и: 333. что она: 332. она не: 298. в тот: 293. у он: 289. с она: 286. она и: 265. и в: 259. с он: 257. это быть: 253. не знать: 253. клайд и: 252. и клайд: 249. он в: 247. 

#### Составлю топ-25 триграмм
То же, что и с биграммами: частотные триграммы - комбинации из частотных слов языка. Похоже. что "озеро большой выпь", какое бы у него ни было первоначальное название, является популярным местом в книге.

In [9]:
trigs = nltk.trigrams(words)
top_trigrams = Counter(trigs).most_common(25)
for pair in top_trigrams:
    print(pair[0][0], pair[0][1], pair[0][2], ':', pair[1], end='. ')

в тот же : 129. в конец конец : 111. во всякий случай : 100. озеро большой выпь : 93. тот же время : 90. до сей пора : 89. он не мочь : 88. и все же : 85. по крайний мера : 82. о то что : 80. в самый дело : 77. белнеп и джефсон : 73. я не мочь : 68. о то как : 64. в этот минута : 63. после то как : 59. для то чтобы : 58. в этот время : 57. на озеро большой : 56. не знать что : 52. то что он : 51. в то что : 47. и в тот : 47. у он быть : 47. бухта третий миля : 45. 

#### Токенизирую и проанализирую оригинальный текст
Делаю это для того, чтобы сохранить полученные данные в файле json. В условиях задания требуется сохранять начальную форму, что потеряло бы смысл, если бы я сохраняла анализ лемматизированных слов.

In [10]:
%%time

not_lemma_words = [w.lower() for w in word_tokenize(text) if w.isalpha()]

not_lemma_analyzed = []
for word in not_lemma_words:
    not_lemma_analyzed.append(morph.parse(word))

forms_data = []
for ana in not_lemma_analyzed:
    forms_data.append({ana[0].word:{'normal_form':ana[0].normal_form, 'Part of Speech':ana[0].tag.POS}})

Wall time: 1min 22s


#### Сохраняю результат предыдущей обработки в файле pymorphy_data.json

In [11]:
with open('pymorphy_data.json', 'w', encoding='utf-8') as f:
    json.dump(forms_data, f, ensure_ascii = False)